In [36]:

# %pip install kagglehub

In [38]:
import math
from pathlib import Path

import pandas as pd
import numpy as np
import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from pprint import pprint

In [2]:
# Download latest version
movie_lens_small_path = kagglehub.dataset_download("shubhammehta21/movie-lens-small-latest-dataset")
print("Path to dataset files:", movie_lens_small_path)

Path to dataset files: C:\Users\Daniel\.cache\kagglehub\datasets\shubhammehta21\movie-lens-small-latest-dataset\versions\1


## Определение метрик для оценки рекомендаций

В этой ячейке задаются функции для расчёта классических ranking‑метрик топ‑N рекомендаций:

- `precision_at_k`, `recall_at_k`, `f1_at_k` — качество рекомендаций по точности и полноте в топ‑K.
- `hit_rate_at_k` — доля пользователей, у которых в топ‑K есть хотя бы один релевантный объект.
- `mrr_at_k` — средний обратный ранг первого релевантного объекта.
- `average_precision_at_k` и `ndcg_at_k` — метрики, учитывающие и факт релевантности, и позицию элемента в выдаче.

Эти функции будут использоваться для сравнения разных рекомендательных подходов.


In [15]:
def precision_at_k(pred_items, true_items, k):
    if k == 0:
        return 0.0
    pred_k = pred_items[:k]
    hit_count = sum(1 for item in pred_k if item in true_items)
    return hit_count / k

def recall_at_k(pred_items, true_items, k):
    if not true_items:
        return 0.0
    pred_k = pred_items[:k]
    hit_count = sum(1 for item in pred_k if item in true_items)
    return hit_count / len(true_items)

def f1_at_k(pred_items, true_items, k):
    p = precision_at_k(pred_items, true_items, k)
    r = recall_at_k(pred_items, true_items, k)
    if p + r == 0:
        return 0.0
    return 2 * p * r / (p + r)

def hit_rate_at_k(pred_items, true_items, k):
    pred_k = pred_items[:k]
    return 1.0 if any(item in true_items for item in pred_k) else 0.0

def mrr_at_k(pred_items, true_items, k):
    pred_k = pred_items[:k]
    for rank, item in enumerate(pred_k, start=1):
        if item in true_items:
            return 1.0 / rank
    return 0.0

def average_precision_at_k(pred_items, true_items, k):
    pred_k = pred_items[:k]
    if not true_items:
        return 0.0
    hits = 0
    sum_precisions = 0.0
    for rank, item in enumerate(pred_k, start=1):
        if item in true_items:
            hits += 1
            sum_precisions += hits / rank
    if hits == 0:
        return 0.0
    return sum_precisions / hits

def dcg_at_k(pred_items, true_items, k):
    pred_k = pred_items[:k]
    dcg = 0.0
    for rank, item in enumerate(pred_k, start=1):
        if item in true_items:
            dcg += 1.0 / math.log2(rank + 1)
    return dcg

def ndcg_at_k(pred_items, true_items, k):
    actual_dcg = dcg_at_k(pred_items, true_items, k)
    ideal_hits = min(len(true_items), k)
    ideal_dcg = sum(1.0 / math.log2(rank + 1) for rank in range(1, ideal_hits + 1))
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

## MovieLens small dataset EDA and Model train

In [5]:
ratings_small = pd.read_csv(Path(movie_lens_small_path) / Path("ratings.csv"))   # userId,movieId,rating,timestamp
movies_small = pd.read_csv(Path(movie_lens_small_path) / Path("movies.csv"))     # movieId,title,genres

ratings_small = ratings_small[["userId", "movieId", "rating"]]

In [30]:
ratings_small.duplicated().sum()

np.int64(0)

## Рекомендации на основе baseline (µ + b_u + b_i)

Здесь baseline‑модель используется уже как рекомендательная:

- Для каждого пользователя собирается множество фильмов, которые он уже смотрел в train.
- Функция `recommend_baseline` вычисляет скор для всех фильмов по формуле µ + b_u + b_i.
- Из списка кандидатов удаляются уже просмотренные фильмы, и возвращается топ‑N фильмов с наибольшим предсказанным рейтингом.


In [16]:
train, test = train_test_split(
    ratings_small,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# глобальное среднее
global_mean = train["rating"].mean()

# item bias и user bias считаем по train
item_mean = train.groupby("movieId")["rating"].mean()
item_bias = item_mean - global_mean

user_mean = train.groupby("userId")["rating"].mean()
user_bias = user_mean - global_mean

def predict_rating_baseline(uid, mid):
    bu = user_bias.get(uid, 0.0)
    bi = item_bias.get(mid, 0.0)
    return float(global_mean + bu + bi)

# RMSE по test
test_pred = test.copy()
test_pred["pred"] = [
    predict_rating_baseline(u, m)
    for u, m in zip(test_pred["userId"], test_pred["movieId"])
]

mse = mean_squared_error(test_pred["rating"], test_pred["pred"])
rmse = float(np.sqrt(mse))
print("Baseline RMSE:", rmse)

Baseline RMSE: 0.9174463561734154


In [18]:
# фильмы, которые пользователь уже оценивал (по train)
train_items_by_user = (
    train.groupby("userId")["movieId"]
    .apply(set)
    .to_dict()
)

def recommend_baseline(user_id, top_n=10):
    """Top-N по baseline-оценке µ + b_u + b_i."""
    bu = user_bias.get(user_id, 0.0)
    # скор для всех фильмов
    scores_s = global_mean + bu + item_bias.reindex(movies_small["movieId"]).fillna(0.0)
    scores_s.index = movies_small["movieId"]

    seen = train_items_by_user.get(user_id, set())
    scores_s = scores_s.drop(labels=list(seen), errors="ignore")

    top_movies = scores_s.sort_values(ascending=False).head(top_n).index
    return list(top_movies)


## Жанровые признаки и профили пользователей (content features)

В этой ячейке готовятся данные для content‑based модели:

- Поле `genres` из `movies_small` преобразуется в one‑hot признаки жанров (матрица фильмов по жанрам).
- Для пользователей строится жанровый профиль: усреднение жанровых векторов всех фильмов с рейтингом ≥ 4.
- Формируется словарь `user_seen` с множеством фильмов, которые каждый пользователь уже смотрел (по всем рейтингам).

Эти данные будут использоваться для рекомендаций по косинусному сходству жанров.

In [19]:
# one-hot по жанрам
movies_genres = movies_small.copy()
genre_ohe = movies_genres["genres"].str.get_dummies(sep="|")
if "(no genres listed)" in genre_ohe.columns:
    genre_ohe = genre_ohe.drop(columns="(no genres listed)")

movie_features = pd.concat(
    [movies_genres[["movieId", "title"]], genre_ohe],
    axis=1
)
movie_features.set_index("movieId", inplace=True)
genre_cols = genre_ohe.columns

# положительные оценки (>= 4) для профиля
positive_ratings = ratings_small[ratings_small["rating"] >= 4.0]

user_movie = positive_ratings.merge(
    movie_features[genre_cols],
    left_on="movieId",
    right_index=True,
    how="inner"
)

user_profile = (
    user_movie.groupby("userId")[genre_cols]
    .mean()
)

# фильмы, которые пользователь уже видел (по всему ratings_small)
user_seen = (
    ratings_small.groupby("userId")["movieId"]
    .apply(set)
    .to_dict()
)


## Content-based рекомендации и оценка моделей

В этой ячейке:

- Формируется множество релевантных фильмов для каждого пользователя в тесте (`rating >= 4`).
- Определяется функция `recommend_content_based`, которая:
  - берёт жанровый профиль пользователя,
  - считает косинусное сходство с жанровыми векторами всех фильмов,
  - исключает уже просмотренные фильмы и возвращает топ‑N по сходству.
- Функция `evaluate_recommender` вычисляет для заданной функции рекомендаций набор top‑K метрик (precision, recall, F1, hit rate, MRR, MAP, NDCG).

Так мы можем в одинаковых условиях сравнить baseline‑подход и content‑based модель по качеству рекомендаций.

In [28]:
# релевантные элементы в test: rating >= 4
test_high = test[test["rating"] >= 4.0]
true_items_by_user = (
    test_high.groupby("userId")["movieId"]
    .apply(set)
    .to_dict()
)

def recommend_content_based(user_id, top_n=10):
    """Top-N content-based по косинусу профиля и жанров."""
    if user_id not in user_profile.index:
        return []

    u_vec = user_profile.loc[user_id][genre_cols].values.reshape(1, -1)
    movie_vecs = movie_features[genre_cols].values

    sims = cosine_similarity(u_vec, movie_vecs)[0]
    scores_s = pd.Series(sims, index=movie_features.index)

    seen = user_seen.get(user_id, set())
    scores_s = scores_s.drop(labels=list(seen), errors="ignore")

    top_movies = scores_s.sort_values(ascending=False).head(top_n).index
    return list(top_movies)

def evaluate_recommender(recommend_fn, K=10):
    users = sorted(true_items_by_user.keys())

    precisions, recalls, f1s = [], [], []
    hits, mrrs, maps, ndcgs = [], [], [], []

    for user_id in users:
        true_items = true_items_by_user.get(user_id, set())
        if not true_items:
            continue

        pred_items = recommend_fn(user_id, top_n=K*5)
        if not pred_items:
            continue

        precisions.append(precision_at_k(pred_items, true_items, K))
        recalls.append(recall_at_k(pred_items, true_items, K))
        f1s.append(f1_at_k(pred_items, true_items, K))
        hits.append(hit_rate_at_k(pred_items, true_items, K))
        mrrs.append(mrr_at_k(pred_items, true_items, K))
        maps.append(average_precision_at_k(pred_items, true_items, K))
        ndcgs.append(ndcg_at_k(pred_items, true_items, K))

    return {
        "precision@10": float(np.mean(precisions)) if precisions else 0.0,
        "recall@10": float(np.mean(recalls)) if recalls else 0.0,
        "f1@10": float(np.mean(f1s)) if f1s else 0.0,
        "hit_rate@10": float(np.mean(hits)) if hits else 0.0,
        "mrr@10": float(np.mean(mrrs)) if mrrs else 0.0,
        "map@10": float(np.mean(maps)) if maps else 0.0,
        "ndcg@10": float(np.mean(ndcgs)) if ndcgs else 0.0,
    }

baseline_metrics = evaluate_recommender(recommend_baseline, K=10)
content_metrics = evaluate_recommender(recommend_content_based, K=10)

## Сводка результатов моделей

Здесь выводятся результаты эксперимента:

- `RMSE_baseline` — качество предсказания рейтингов baseline‑моделью.
- `baseline` — набор top‑10 метрик для рекомендаций на основе µ + b_u + b_i.
- `content` — тот же набор метрик для content‑based модели по жанрам.

Эти значения позволяют сравнить оба подхода как по RMSE, так и по качеству топ‑N рекомендаций.

In [29]:
pprint(
    {
    "RMSE_baseline": rmse,
    "baseline": baseline_metrics,
    "content": content_metrics,
    }
)

{'RMSE_baseline': 0.9174463561734154,
 'baseline': {'f1@10': 0.0001638029778150318,
              'hit_rate@10': 0.005008347245409015,
              'map@10': 0.0022815804117974404,
              'mrr@10': 0.0022815804117974404,
              'ndcg@10': 0.0006404560148887755,
              'precision@10': 0.0005008347245409016,
              'recall@10': 0.00010085167679453125},
 'content': {'f1@10': 0.0,
             'hit_rate@10': 0.0,
             'map@10': 0.0,
             'mrr@10': 0.0,
             'ndcg@10': 0.0,
             'precision@10': 0.0,
             'recall@10': 0.0}}
